<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# SQL Exercises
_**Author**: Boom D. (DSI-NYC) & Noah C. (DSIR-1019)_ 
***

## Conceptual Interview Questions

**1. Explain the difference between INNER JOIN, LEFT JOIN, and RIGHT JOIN**

- **INNER JOIN** takes all rows that exist in both tables (the intersection) based on the joined column key.
- **LEFT JOIN** takes all rows from the LEFT table so the number of rows must be equal to the left table's. So this includes the intersection, but also any unique records to the left table. If you have selected some columns from the RIGHT table, but there are no related records in the left table, these records will show NULL values.
- **RIGHT JOIN** is like the above but gets all records in the RIGHT table.

There are some other JOIN types if you're interested. Here's a nice infographic:
<img src = "http://www.postgresqltutorial.com/wp-content/uploads/2018/12/PostgreSQL-Joins.png" style="float: left; margin: 20px; height: 300px">

**2. Explain the difference between GROUP BY, WHERE, and HAVING**
- `GROUP BY` is used in conjunction with aggregate functions (e.g. SUM(), COUNT(), etc.) to specify 
- `WHERE` is used as a filter on fields (e.g. date, name, etc.)
- `HAVING` is used as a filter on aggregate functions (e.g. SUM(), COUNT(), etc.)

Examples here: https://www.geeksforgeeks.org/sql-group-by/

## `WHERE` Review Exercises

**3. Countries** <br>
_(Source: https://www.codewars.com/kata/on-the-canadian-border-sql-for-beginners-number-2/sql)_

Select names, and countries of origin of all the travelers, excluding anyone from Canada, Mexico, or The US.

_Answer:_
```MySQL
SELECT name,country
FROM travelers
WHERE country NOT IN ('Canada','Mexico','USA')
```

**4. Harry Potter** <br>
_(Source:https://www.codewars.com/kata/sql-with-harry-potter-sorting-hat-comparators/sql)_

There is truly no magic in the world; the Hogwarts Sorting Hat is SQL-based, its decision-making powers are common operators and prospectIve students are merely data - names, and two columns of qualities.

`students` schema
- id
- name
- quality1
- quality2

Note that:
- Slytherin are being quite strict this year and will only take students who are evil AND cunning.
- Gryffindor will take students who are brave but only if their second quality is NOT evil.
- Ravenclaw accepts students who are studious OR intelligent.
- Hufflepuff will simply take those who have the quality hufflepuff.

_(Don't worry, for simplicity's sake 'brave' and 'studious' will only appear in quality1, and 'cunning' and 'intelligent' will only appear in quality2.)_

Return the id, name, quality1 and quality2 of all the students who'll be accepted, ordered by ascending id.

_Answer:_

```MySQL
SELECT s.id, s.name, s.quality1, s.quality2
from students as s
WHERE (s.quality1 = 'evil' AND s.quality2 = 'cunning') -- Slytherin
      OR (s.quality1 = 'brave' AND s.quality2 != 'evil') -- Gryfindor
      OR (s.quality1 = 'studious' OR s.quality2 = 'intelligent') -- Ravenclaw
      OR (s.quality1 = 'hufflepuff' OR s.quality2 = 'hufflepuff') -- Hufflepuff
ORDER BY s.id ASC
```

## SQL with the Noble Houses of Westeros

For all of the following questions, refer to the datasets found in the `got_data` folder.

In [2]:
import pandas as pd
from pandasql import sqldf

In [3]:
bannermen = pd.read_csv('got_data/bannermen.csv')
houses = pd.read_csv('got_data/houses.csv')
people = pd.read_csv('got_data/people.csv')

**5. Targaryens are very fond of the prefix "Ae-" in their names.**

Write a SQL query that returns the first name and houses of all characters whose first name begins with the prefix "Ae-"

In [8]:
sql = """
SELECT c.first_name, c.house
FROM people AS c
WHERE c.first_name LIKE 'Ae%'
"""

df = sqldf(sql)
df.head()

,first_name,house
0,aegon,targaryen
1,aerys,targaryen


_Answer:_
```SQL
SELECT c.first_name, c.house
FROM people AS c
WHERE c.first_name ILIKE 'Ae%'  -- can use ILIKE instead of LIKE to ignore case sensitivity
```

**6. As the Ironborn say in Pyke, "what is dead may never die".**

Write a SQL query that returns the first name, nickname, and living status of all members of the ruling family of Pyke.

_Answer:_
```MySQL
SELECT c.first_name, c.nickname, c.alive
FROM people AS c
    INNER JOIN houses AS h ON c.house = h.name
WHERE h.domain = 'pyke'
```

In [9]:
sql = """
SELECT c.first_name, c.nickname, c.alive
FROM people AS c
    INNER JOIN houses AS h ON c.house = h.name
WHERE h.domain = 'pyke'
"""

df = sqldf(sql)
df.head()

,first_name,nickname,alive
0,theon,reek,0
1,yara,None,1


**7. Queen Cersei demands to know who the vassal houses (bannermen) of Houses Targaryen, Tully, Tyrell, Baratheon are.**

Write a SQL query that displays the `name` of the bannermen (as "vassal house") of Houses Targarygen, Tully, Tyrell, and Baratheon, along with the `size` of their armies. This should be ordered by the 4 leader houses.

_Answer:_
```MySQL
SELECT  h.name AS "leader house",
		b.name AS "vassal house",
        b.size
FROM bannermen AS b
    INNER JOIN houses AS h ON b.leader_house_id = h.id
WHERE h.name IN ('targaryen', 'tully', 'tyrell', 'baratheon')
ORDER BY    "leader house" ASC,
            b.size DESC,        -- optional
            "vassal house" ASC  -- optional
```

In [10]:
sql = """
SELECT  h.name AS "leader house",
        b.name AS "vassal house",
        b.size
FROM bannermen AS b
    INNER JOIN houses AS h ON b.leader_house_id = h.id
WHERE h.name IN ('targaryen', 'tully', 'tyrell', 'baratheon')
ORDER BY    "leader house" ASC,
            b.size DESC,        -- optional
            "vassal house" ASC  -- optional
"""

df = sqldf(sql)
df.head()

,leader house,vassal house,size
0,baratheon,selmy,40
1,baratheon,tarth,20
2,tully,blackwood,25
3,tully,whent,25
4,tyrell,hightower,25


**8. Eddard is helping Robert gather up their troops. A leader house's force is equal to the sum of the sizes of all bannermen houses who have sworn fealty to them.**

However, due to a certain erm...head injury... Eddard seems to have forgotten which noble houses have sworn fealty to his and Robert's houses. Eddard's son Brandon uses his Greensight to travel to the future and reaches out to you for help.

(a) **_Easier Version:_** Write a SQL query that returns the value of the force of Eddard's house.

_Answer:_ 140, which we'll get from the following query

```MySQL
SELECT SUM(b.size)
FROM bannermen AS b
    INNER JOIN houses AS h ON b.leader_house_id = h.id
    INNER JOIN people AS c ON h.name = c.house
WHERE c.first_name = 'eddard'
```

In [11]:
sql = """
SELECT SUM(b.size)
FROM bannermen AS b
    INNER JOIN houses AS h ON b.leader_house_id = h.id
    INNER JOIN people AS c ON h.name = c.house
WHERE c.first_name = 'eddard'
"""

df = sqldf(sql)
df.head()

,SUM(b.size)
0,140


(b) _**Harder Version:**_ Write a SQL query that returns the value of the force (call it "force") for each of Eddard's and Robert's houses (call it "leader_house").

```MySQL
SELECT h.name AS "leader_house",
       SUM(b.size) AS "force"
FROM houses AS h
    INNER JOIN bannermen AS b ON h.id = b.leader_house_id
    INNER JOIN people AS c ON h.name = c.house
WHERE c.first_name IN ('eddard', 'robert')
GROUP BY leader_house
```

In [12]:
sql = """
SELECT h.name AS "leader_house",
       SUM(b.size) AS "force"
FROM houses AS h
    INNER JOIN bannermen AS b ON h.id = b.leader_house_id
    INNER JOIN people AS c ON h.name = c.house
WHERE c.first_name IN ('eddard', 'robert')
GROUP BY leader_house
"""

df = sqldf(sql)
df.head()

,leader_house,force
0,baratheon,60
1,stark,140


## Harder CodeWars Questions on `JOIN`, `GROUP BY`, `SUM`, etc.

**9. Pokemon:** <br>
_(Source: https://www.codewars.com/kata/sql-with-pokemon-damage-multipliers/)_

You have arrived at the Celadon Gym to battle Erika for the Rainbow Badge. She will be using Grass-type Pokemon. Any fire pokemon you have will be strong against grass, but your water types will be weakened. The multipliers table within your Pokedex will take care of that.

`pokemon` schema:
- id
- pokemon_name
- element_id
- str

`multipliers` schema:
- id
- element
- multiplier

Using the above tables, return the pokemon_name, modifiedStrength and element of the Pokemon whose strength, after taking these changes into account, is greater than or equal to 40, ordered from strongest to weakest.

_Answer:_ 
```MySQL
SELECT pkmn.pokemon_name,
       pkmn.str*mult.multiplier as modifiedStrength,
       mult.element
FROM pokemon as pkmn 
    INNER JOIN multipliers as mult ON pkmn.element_id = mult.id
WHERE pkmn.str*mult.multiplier >= 40
ORDER BY modifiedStrength DESC  
```

**10. Street Fighter** <br>
(_Source: https://www.codewars.com/kata/sql-with-street-fighter-total-wins/_)

Each row of the table fighters records, alongside the fighter's name, whether they won (1) or lost (0), as well as the type of move that ended the bout.

`fighters` schema:
- id
- name
- won
- lost
- move_id
- winning_moves

`move` schema:
- id
- move

However, due to new health and safety regulations, all ki blasts have been outlawed as a potential fire hazard. Any bout that ended with Hadoken, Shouoken or Kikoken should not be counted in the total wins and losses.

_YOUR TASK:_
- Return name, won, and lost columns displaying the name, total number of wins and total number of losses. Group by the fighter's name.
- Do not count any wins or losses where the winning move was Hadoken, Shouoken or Kikoken.
- Order from most-wins to least
- Return the top 6. Don't worry about ties.

_Answer:_
    
```MySQL
SELECT  f.name,
        sum(f.won) as won,
        sum(f.lost) as lost
FROM fighters f
    INNER JOIN winning_moves m ON f.move_id = m.id
WHERE m.move NOT IN ('Hadoken', 'Shouoken', 'Kikoken')
GROUP BY f.name
ORDER BY won DESC
LIMIT 6
```